In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import wfdb
import os
import scipy
from scipy.signal import butter, filtfilt
from sklearn.preprocessing import StandardScaler

In [8]:
#import data and define hyperparameters
output_dir = 'mitdb'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Direktori '{output_dir}' berhasil dibuat.")

In [9]:
print(f"Mendownload record dari database 'mitdb' ke '{output_dir}'...")
wfdb.dl_database('mitdb', annotators=['atr'], dl_dir=output_dir)
print("Download selesai!")

record_path = os.path.join(output_dir, '100')
record = wfdb.rdrecord(record_path, sampto=3000) # Ambil 3000 sampel pertama
annotation = wfdb.rdann(record_path, 'atr', sampto=3000)

signal = record.p_signal[:, 0] # Ambil sinyal dari channel pertama aja
sfreq = record.fs # Frekuensi sampling

print(f"\nSinyal berhasil dimuat. Frekuensi sampling: {sfreq} Hz")
print(f"Panjang sinyal: {len(signal)} sampel")

Mendownload record dari database 'mitdb' ke 'mitdb'...
Generating record list for: 100
Generating record list for: 101
Generating record list for: 102
Generating record list for: 103
Generating record list for: 104
Generating record list for: 105
Generating record list for: 106
Generating record list for: 107
Generating record list for: 108
Generating record list for: 109
Generating record list for: 111
Generating record list for: 112
Generating record list for: 113
Generating record list for: 114
Generating record list for: 115
Generating record list for: 116
Generating record list for: 117
Generating record list for: 118
Generating record list for: 119
Generating record list for: 121
Generating record list for: 122
Generating record list for: 123
Generating record list for: 124
Generating record list for: 200
Generating record list for: 201
Generating record list for: 202
Generating record list for: 203
Generating record list for: 205
Generating record list for: 207
Generating record

In [12]:
def bandpass_filter(signal, lowcut=0.5, highcut=40.0, fs=360, order=1):
    nyq = 0.5 * fs
    b, a = butter(order, [lowcut / nyq, highcut / nyq], btype='band')
    return filtfilt(b, a, signal)

filtered_signal = bandpass_filter(signal, fs=sfreq)

In [23]:
r_peaks = annotation.sample
window_before = int(0.3 * sfreq)
window_after = int(0.4 * sfreq)

beats = []
for r in r_peaks:
    if r - window_before >= 0 and r + window_after < len(filtered_signal):
        beat = filtered_signal[r - window_before : r + window_after]
        beats.append(beat)
beats = np.array(beats)


In [24]:
scaler = StandardScaler()
beats_normalized = scaler.fit_transform(beats)

In [25]:
label_mapping = {
    'N': 0, 'L': 0, 'R': 0,  # Normal beats
    'A': 1,                  # Atrial premature
    'V': 2, 'F': 2,
    '/': 3          # Ventricular-related
    # Tambahkan label lain sesuai kebutuhan
}

labels = [label_mapping.get(sym, -1) for sym in annotation.symbol]
beats_filtered, labels_filtered = [], []
for b, l in zip(beats_normalized, labels):
    if l != -1:
        beats_filtered.append(b)
        labels_filtered.append(l)


In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    np.array(beats_filtered),
    np.array(labels_filtered),
    test_size=0.2,
    random_state=42
)

# Reshape biar cocok untuk LSTM (samples, timesteps, 1)
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]

np.save("dataset/X_train.npy", X_train)
np.save("dataset/y_train.npy", y_train)
np.save("dataset/X_test.npy", X_test)
np.save("dataset/y_test.npy", y_test)
